<a href="https://colab.research.google.com/github/fatemafaria142/Evaluating-LLMs-Generated-Food-Recipes-in-Comparison-to-Established-Reference-Recipes/blob/main/Recipe_Generation_using_Mistral_7B_Instruct_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [2]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


#### **Dataset Link:** https://huggingface.co/datasets/recipe_nlg

In [3]:
import pandas as pd

# Load the original dataset
dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/full_dataset.csv")

# Extract the first 3000 rows
subset_dataset = dataset.head(3000)

# Save the subset to a new CSV file
subset_dataset.to_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation_subset.csv", index=False)


In [4]:
import pandas as pd
instruct_tune_dataset = pd.read_csv("/content/drive/MyDrive/Recipe Generation/recipe_generation_subset.csv")
instruct_tune_dataset.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [5]:
length_of_data = len(instruct_tune_dataset)
# Print the length
print(length_of_data)

3000


In [6]:
from datasets import Dataset, DatasetDict

# Create a DatasetDict for instruct_tune_dataset
instruct_tune_dataset = DatasetDict()

# Select 1500 rows for the 'train' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["train"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].head(2500))

# Select 200 rows for the 'test' split and only keep 'ingredients' and 'directions'
instruct_tune_dataset["test"] = Dataset.from_pandas(dataset[["ingredients", "directions"]].tail(200))

# Print the updated instruct_tune_dataset
print(instruct_tune_dataset)

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})


### **We will use just a small subset of the data for this training example**

In [7]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['ingredients', 'directions'],
        num_rows: 200
    })
})

* Note that this time, the tokenizer has added the control tokens `[INST]` and `[/INST]` to indicate the start and end of user messages (but not assistant messages!). **Mistral-instruct was trained with these tokens.**
* In order to leverage instruction fine-tuning, your prompt should be surrounded by `[INST]` and `[/INST]` tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

In [9]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    eos_token = "</s>"

    # Use a predefined template for instructions
    instructions_template = "Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "



    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "[INST]"
    full_prompt += instructions_template
    full_prompt += sample['ingredients']
    full_prompt += "[/INST]"
    full_prompt += sample['directions']
    full_prompt += eos_token

    return full_prompt

In [10]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"][/INST]["In a heavy 2-quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine.", "Stir over medium heat until mixture bubbles all over top.", "Boil and stir 5 minutes more. Take off heat.", "Stir in vanilla and cereal; mix well.", "Using 2 teaspoons, drop and shape into 30 clusters on wax paper.", "Let stand until firm, about 30 minutes."]</s>'

In [11]:
create_prompt(instruct_tune_dataset["train"][1])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton sour cream"][/INST]["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\\u00b0 for 3 hours."]</s>'

In [12]:
create_prompt(instruct_tune_dataset["train"][2])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg. cream cheese, cubed", "1/3 c. butter, cubed", "1/2 tsp. garlic powder", "1/2 tsp. salt", "1/4 tsp. pepper"][/INST]["In a slow cooker, combine all ingredients. Cover and cook on low for 4 hours or until heated through and cheese is melted. Stir well before serving. Yields 6 servings."]</s>'

In [13]:
create_prompt(instruct_tune_dataset["train"][3])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"][/INST]["Boil and debone chicken.", "Put bite size pieces in average size square casserole dish.", "Pour gravy and cream of mushroom soup over chicken; level.", "Make stuffing according to instructions on box (do not make too moist).", "Put stuffing on top of chicken and gravy; level.", "Sprinkle shredded cheese on top and bake at 350\\u00b0 for approximately 20 minutes or until golden and bubbly."]</s>'

In [14]:
create_prompt(instruct_tune_dataset["train"][4])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 c. peanut butter", "3/4 c. graham cracker crumbs", "1 c. melted butter", "1 lb. (3 1/2 c.) powdered sugar", "1 large pkg. chocolate chips"][/INST]["Combine first four ingredients and press in 13 x 9-inch ungreased pan.", "Melt chocolate chips and spread over mixture. Refrigerate for about 20 minutes and cut into pieces before chocolate gets hard.", "Keep in refrigerator."]</s>'

In [15]:
create_prompt(instruct_tune_dataset["train"][5])

'<s>[INST]Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"][/INST]["Wash potatoes; prick several times with a fork.", "Microwave them with a wet paper towel covering the potatoes on high for 6-8 minutes.", "The potatoes should be soft, ready to eat.", "Let them cool enough to handle.", "Cut in half lengthwise; scoop out pulp and reserve.", "Discard shells.", "Brown ground beef until done.", "Drain any grease from the meat.", "Set aside when done.", "Meat will be added later.", "Melt butter in a large kettle over low heat; ad

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [17]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [18]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [19]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [20]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [21]:
# Use a predefined template for instructions
prompt = "<s>[INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: "
prompt += '''["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] [/INST]'''
print(prompt)

<s>[INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] [/INST]


In [22]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s><s> [INST] Imagine you\'re a chef, and you\'ve got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] [/INST] Title: Loaded Baked Potatoes Casserole\n\nIngredients:\n- 6 baking potatoes, washed and scrubbed\n- 1 lb. extra lean ground beef\n- 2/3 cup butter or margarine\n- 6 cups milk\n- 3/4 tsp. salt\n- 1/2 tsp. pepper\n- 1 1/2 cups shredded Cheddar cheese, divided\n- 12 sliced bacon, cooked, crumbled, and divided\n- 4 green onions, chopped and divided\n- 1 (8 oz.) carton sour cream (optional)\n\nInstructions:\n\n1. Preheat the oven to 375°F (190°C).\n2. Bake t

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [23]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [24]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [26]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [27]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [28]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.535200
20,1.111200
30,1.003300
40,0.920300
50,0.927100
60,0.911100
70,0.834500
80,0.831600
90,0.834300
100,0.821300


TrainOutput(global_step=100, training_loss=0.9729870510101318, metrics={'train_runtime': 1562.6257, 'train_samples_per_second': 0.256, 'train_steps_per_second': 0.064, 'total_flos': 1.74835334381568e+16, 'train_loss': 0.9729870510101318, 'epoch': 0.78})

### **Save the model**

In [29]:
trainer.save_model("mistral_instruct_generation")

In [30]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [31]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [32]:
# Use a predefined template for instructions
prompt = '''<s>[INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: '''
prompt += '''["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"] [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s><s> [INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 large whole chicken", "2 (10 1/2 oz.) cans chicken gravy", "1 (10 1/2 oz.) can cream of mushroom soup", "1 (6 oz.) box Stove Top stuffing", "4 oz. shredded cheese"] [/INST] Title: Chicken and Gravy Casserole

Prepare the following steps to cook this scrumptious dish:

1. Preheat oven to 350°F (180°C).
2. Rinse the whole chicken under cold running water. Drain and set aside.
3. In a baking dish, place the chicken.
4. Pour one can of chicken gravy over the chicken.
5. Spread half of the can of cream of mushroom soup on top of the gravy layer.
6. Mix the Stove Top stuffing according to the package instructions and spread it over the soup.
7. Add the remaining can of cream of mushroom soup on top of the stuffing.
8. Sprinkle shredded cheese over the top of the casserole.
9. 

### **Example No:2**

In [33]:
# Example usage
prompt = '''[INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: '''
prompt += '''["1 can cream-style corn", "1 can whole kernel corn", "1/2 pkg. (approximately 20) saltine crackers, crushed", "1 egg, beaten", "6 tsp. butter, divided", "pepper to taste"][/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)


<s> [INST] Imagine you're a chef, and you've got these delicious ingredients in front of you. Now, all you need are to give the directions on how to cook up a mouthwatering dish. Here is the list of ingredients: ["1 can cream-style corn", "1 can whole kernel corn", "1/2 pkg. (approximately 20) saltine crackers, crushed", "1 egg, beaten", "6 tsp. butter, divided", "pepper to taste"][/INST] First, preheat your oven to 350 degrees Fahrenheit.

Next, empty the cans of cream-style corn and whole kernel corn into a 2-quart baking dish and set it aside.

Now, take the crushed saltine crackers and distribute them evenly over the top of the corn.

Melt 4 teaspoons of butter in a small saucepan over medium heat and set it aside as well.

In a small bowl, beat the egg and mix in the melted butter. Pour the egg mixture over the corn and crackers.

Dot the top with the remaining 2 teaspoons of butter.

Bake in the preheated oven for about 30 minutes, or until the top is golden brown.

Season with p

### **Example No:3**

In [34]:
# Example usage
prompt = '''[INST] Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: '''
prompt += '''["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)


<s> [INST] Put yourself in the shoes of a thoughtful mental health counselor. Imagine you're all set to help with kindness and understanding, ready to offer care and support. Your task is to extend understanding and guidance, providing support to individuals dealing with mental health challenges. Here is my mental health problems: ["6 baking potatoes", "1 lb. of extra lean ground beef", "2/3 c. butter or margarine", "6 c. milk", "3/4 tsp. salt", "1/2 tsp. pepper", "1 1/2 c (6 oz.) shredded Cheddar cheese, divided", "12 sliced bacon, cooked, crumbled and divided", "4 green onion, chopped and divided", "1 (8 oz.) carton sour cream (optional)"] [/INST] I'm really sorry to hear that you're dealing with mental health challenges, and I want you to know that I'm here to listen and offer support in any way I can. The fact that you've shared a list of ingredients with me suggests that you might be dealing with feelings of anxiety or stress related to cooking or meal preparation.

It's important